In [ ]:
class DatabaseConnection:
    def __init__(self):
        pass
    
    def connection_properties(self, environment):
        if environment == 'production':
            print(prod)
        else:
            pass
        
        jdbcurl = "jdbc:sqlserver://{}:{};database={};authentication=ActiveDirectoryServicePrincipal;encrypt=true;trustServerCertificate=false;HostNameInCertificate=*.database.windows.net;loginTimeout=30;AADSecurePrincipalId={}@{};AADSecurePrincipalSecret={}".format(
            host,
            port,
            database,
            client_id,
            tenant_id,
            client_secret
        )
        
        connectionProperties = {
            "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
        }
        
        return {"url": jdbcurl, "properties": connectionProperties}
    
    def read_data_sqldb(self, read_query, environment):
        db_connection = self.connection_properties(environment)
        try:
            print("Reading query...")
            df_read_query = spark.read.jdbc(
                url=db_connection["url"],
                table=f"({read_query}) AS temp_table",
                properties=db_connection["properties"]
            )
            return df_read_query
        except Exception as exe:
            print("Running exception")
            error = type(exe).__name__
            error_des = str(exe)
            print(f"Error: {error}")
            print(f"Error Description: {error_des}")
            return None

# Usage
conn = DatabaseConnection()
df = conn.read_data_sqldb("SELECT 'test-a1' AS name1, 'test-a2' AS name2", "nonprod")